In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [61]:
dists = pd.read_csv("data/atlas_open_data_files/distributions.csv")
dist_changes = pd.read_csv("data/atlas_open_data_files/distribution_changes.csv")
grid_coords = pd.read_csv("data/atlas_open_data_files/grid_square_coordinates_lookup.csv")
pct_bench_species = pd.read_csv("data/atlas_open_data_files/percent_benchmark_species_detected.csv")

species_lookup = pd.read_csv("data/atlas_open_data_files/species_lookup_nocase.csv", encoding="ISO-8859-1")

#### Misc stuff

In [33]:
# The number of atlases in the dataset
dists["period"].unique()

array(['1968-72', '2008-11', '2007/08-10/11', '1980/81-1982/83',
       '1988-91'], dtype=object)

In [34]:
# These are the top 10 birds which were spotted most in the Big Garden 
# Birdwatch (2023) https://www.rspb.org.uk/whats-happening/big-garden-birdwatch
HOUSE_SPARROW = 459
BLUE_TIT = 436
STARLING = 457
WOODPIDGEON = 270
BLACKBIRD = 371
ROBIN = 345
GOLDFINCH = 471
GREAT_TIT = 437
MAGPIE = 450
LONG_TAILED_TIT = 431

# Ideas
- Regression of distribution changes of common garden birds?
- Classifying birds that need special concern?
- To what extent does the presence/movement of passerines affect the movement of birds of prey?
    - Regression where y = bird of prey and x = passerine

- **Feed data in to NN which predicts whether a bird will be on the latest Red List or not**

# Predicting conservation status using citizen science bird atlases

In [60]:
# Add species code to redlist.csv by looking up the name in species lookup
redlist = pd.read_csv("data/redlist.csv")
redlist = redlist.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
redlist_ids = pd.merge(redlist, species_lookup, on="english_name", how="left")
redlist_ids

,english_name,prev_status,speccode,scientific_name,taxonomic_rank
0,grey partridge,NaN,131.0,perdix perdix,species
1,lapwing,NaN,171.0,vanellus vanellus,species
2,grasshopper warbler,NaN,384.0,locustella naevia,species
3,ptarmigan,g,126.0,lagopus muta,species
4,whimbrel,NaN,202.0,numenius phaeopus,species
...,...,...,...,...,...
65,yellowhammer,NaN,514.0,emberiza citrinella,species
66,dotterel,NaN,166.0,charadrius morinellus,species
67,marsh warbler,NaN,392.0,acrocephalus palustris,species
68,ringed plover,NaN,160.0,charadrius hiaticula,species
